In [ ]:
# 0
# if you haven't installed wget, uncomment the below command and run this cell
#!pip install wget

In [65]:
# 1
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import time
import wget
import os

In [66]:
# 2
####
# where are you scraping?
# urls = "https://guides.library.fresnostate.edu/agbs120" # if using individual website

## if list of URLs
# Open the file in read mode
with open('libguides-url-list.txt', 'r') as file:
    urls = file.read()
file.close()    
urls = urls.split('\n')


####
# where do you want to save locally?

# save_loc is redundant, but a reminder that you should have locally folders that reflect the years, because that's where below is saving to
save_loc = "libguides" 

# what kind of prefix do you want on your files
save_prefix = "libguides_" 

In [159]:
urls_saved = [] 

In [ ]:
for url in urls:
    if url in urls_saved:
        print("in saved, do nothing1")
    elif url=="":
        print("url is empty")
    else:
        webpage = requests.get(url)
        soup = BeautifulSoup(webpage.content, 'html.parser')
        for link in soup.find_all('a'):
            data_target = link.get('href')
        #        print(data_target)
            time.sleep(.25) # be kind, don't look like a DDOS attack
            if not(data_target):
                print("url is empty")
            else: 
                if ("guides.library.fresnostate.edu") in data_target:
                #        print("is libguide", data_target)
                    if data_target[-1]=="/":
                #            print("strip char")
                        data_target = data_target[:-1]
                            
                    if data_target in urls_saved:
                        print("in saved, do nothing2") 
                    else:
                        if ((".php" in data_target) and not("c.php" in data_target)) or ("/database/" in data_target) or ("/az/" in data_target):  
                            print("some other type of page, profile/subject/database page, do nothing:", data_target)
                        else:
                            urls_saved.append(data_target)
                            wget_url = data_target
                            wget_save = "./libguides/"+save_prefix+data_target.replace("https://guides.library.fresnostate.edu","").replace("/c.php?g=","").replace("/","").replace("&p=","p_")
                            if not((wget_save[-3:]=="htm") or (wget_save[-4:]=="html")):
                                wget_save = wget_save+".html"
                            
                            if os.path.exists(wget_save):
                                print(wget_save+" already exists. Did NOT save.")
                            else:
                                r = requests.head(data_target)
                                if (r.status_code == 200):
                                    print("SAVING "+ wget_url+ " at local location: "+wget_save)
                                    wget.download(wget_url,wget_save)
                                    time.sleep(.25) # be kind, don't look like a DDOS attack

print("fin.")